In [ ]:
from ddpg_reacher import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
env = gym.make("modified_gym_env:ReacherPyBulletEnv-v1", rand_init=False)
ddpg = DDPG(env, action_dim=2, state_dim=8, device=device, critic_lr=1e-3, actor_lr=1e-4, gamma=0.99, batch_size=100)

In [ ]:
# Train the policy
value_losses, policy_losses, validation_reward, validation_steps = ddpg.train(1e5)

In [ ]:
plotting(validation_reward, "Average Rewards for Evaluation",
                            "eval_return.{}.png".format(SEED),"Iterations", "Reward")
plotting(validation_steps, "Steps to Completion",
                            "eval_steps_{}.png".format(SEED),"Iterations", "Steps to Completion")

torch.save(ddpg.actor.model,"./Actor.pth")
torch.save(ddpg.critic.model,"./Critic.pth")

np.save("validation_reward_{}.npy".format(SEED), validation_reward)
np.save("validation_steps_{}.npy".format(SEED), validation_steps)

In [ ]:
# Evaluate the final policy
state, step, done = env.reset(), 0, False
env.render()
while not done:
    state = torch.from_numpy(state).float().unsqueeze(0).to(device)
    action = ddpg.actor(state).detach().squeeze().numpy()
    next_state, reward, done, _ = env.step(action)
    env.render()
    time.sleep(0.1)
    state = next_state
    step+=1
    print("Steps: {}, Action: {}, Reward: {}".format(step, action, reward))